<a href="https://colab.research.google.com/github/4en1x/diploma/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install gcc-4.8 g++-4.8 -y -qq;
!ln -s /usr/bin/gcc-4.8 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-4.8 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.8 60 --slave /usr/bin/g++ g++ /usr/bin/g++-4.8

127 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2019-03-26 07:51:37--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.182.215
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.182.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?0XqmNa5rSfDeMP0jtGScOrzdpzCFH_NIf6xHKa7-g6xnc0JH3_tGuH_6UD0aiz_A_EkWLqd-0ykw23HuGv-EcMFjnSuNEl2xCPFD10OJ-jxundJCHtt3bwGFzm_JvOhO0JAa45EziA_cuIrwpe2BMI4G9X8ilnHYuBL4BJKREZzTUuq3k0jOZRvbASSw0jwDOya6a6zBOxmlRLTeAs6ZGmXqkw [following]
--2019-03-26 07:51:39--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?0XqmNa5rSfDeMP0jtGScO

In [0]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-prl3e8q5
  Stored in directory: /tmp/pip-ephem-wheel-cache-mzzfzupu/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [0]:
!cd /usr/src
!ln -sfn linux-headers-2.6.35-28-generic linux

In [7]:
%%cu
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include "linux/kernel.h"
#include <iostream>
#include <ctime>
#include <iomanip>
#include <fstream>
#include <math.h>
#include <cmath>

using namespace std;

#define ITERATIONS 2000000
#define N 256

#define e 2.7182818

#define blockZSize 3

#define blockXSize 32
#define blockYSize 32

#define tileXSize 4
#define tileYSize 4

#define extN (N + 2 * blockZSize)
#define Dx 1
#define h ((float) Dx / (N + 1))
#define h2 powf(h, 2.0)

__device__
float f(int i, int j) {
	return 2 * powf(e, i * h + j * h);
}

float u_t(float x) {
	return powf(e, x);
}

float u_l(float x) {
	return powf(e, x);
}

float u_r(float x) {
	return powf(e, x + 1);
}

float u_d(float x) {
	return powf(e, x + 1);
}


__device__
void tile(float* matrix, int iGl, int jGl, bool isBlack) {
	int iBegin = iGl * blockXSize + threadIdx.x * tileXSize + blockZSize;
	int jBegin = jGl * blockYSize + threadIdx.y * tileYSize + blockZSize;

  int N1 = tileXSize + blockZSize * 2;
  int N2 = tileYSize + blockZSize * 2;
  
	float helper[10 * 10];
    
  for(int i = 0; i < N1; i++) {
		for(int j = 0; j < N2; j++) {
			helper[i * N1 + j] = matrix[extN * (iBegin - blockZSize + i) + jBegin - blockZSize + j];
		}
	}
  
  int iSplit = iBegin - 2 * blockZSize + 1;
  int jSplit = jBegin - 2 * blockZSize + 1;
    
  for(int k = 1; k < blockZSize; k++) {
		for (int i = k; i < N1 - k; i++) {
      int offset = (i + blockZSize - k) % 2;
		  if (isBlack) {
			  offset = 1 - offset;
		  }
        
		  for (int j = k + offset; j < N2 - k; j += 2) {
        int b = N1 * i + j;
			  helper[b] = 0.25 * (
            helper[b + N1]
          + helper[b - N1]
          + helper[b + 1]
				  + helper[b - 1] 
          - h2 * f(iSplit + i, jSplit + j)
        );
		  }
	  }
	}

  //printf("iGl : %d, %d, %d, %d, %d, %d, %d \n", iGl, jGl, threadIdx.x, threadIdx.y, isBlack, iBegin, jBegin);

	for (int i = iBegin; i < iBegin + tileXSize; i++) {
		int offset = i % 2;
		if (isBlack) {
			offset = 1 - offset;
		}
		for (int j = jBegin + offset; j < jBegin + tileYSize; j += 2) {
      int helperIndex = N1 * (i - iBegin + blockZSize) + j - jBegin + blockZSize;
			matrix[extN * i + j] = 0.25 * (
          helper[helperIndex + N1]
        + helper[helperIndex - N1]
        + helper[helperIndex + 1]
				+ helper[helperIndex - 1] 
        - h2 * f(i - blockZSize + 1, j - blockZSize + 1)
      );
		}
	}
}

__global__
void calculateBlack(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, true);
}

__global__
void calculateRed(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, false);
}

float func(int i, int j) {
	return powf(e, i * h + j * h);
}

float* createMatrix(int row, int col) {
	float* matrix = new float[row * col];

	for(int i = 0; i < row; i++) {
		for(int j = 0; j < col; j++) {
			matrix[i * row + j] = 0;
		}
	}

  for(int k = 0; k < blockZSize; k++) {
    for(int i = k; i < row - k; i++) {
      matrix[k * row + i] = u_t((i - blockZSize + 1) * h);
      matrix[(row - 1 - k) * row + i] = u_d((i - blockZSize + 1) * h);
    }
	}

  for(int k = 0; k < blockZSize; k++) {
    for(int j = k; j < col - k; j++) {
      matrix[j * col + k] = u_l((j - blockZSize + 1) * h);
      matrix[j * col + row - 1 - k] = u_r((j - blockZSize + 1) * h);
	  }
  }
    
	return matrix;
}

float calculateDifference(float* matrix1) {
	float tolerance = 0.0f;
  int tolerancei = 0;
  int tolerancej = 0;
  for (int i = blockZSize; i < extN - blockZSize; i++) {
		for (int j = blockZSize; j < extN - blockZSize; j++) {
      float fCurrent = func(i - blockZSize + 1, j - blockZSize + 1);
      if(abs(matrix1[i * extN + j] - fCurrent) > tolerance) {
          tolerance = abs(matrix1[i * extN + j] - fCurrent);
          tolerancei = i;
          tolerancej = j;
      } 
		}
	}
  //cout << " i: " << tolerancei << "j: " << tolerancej;
	return tolerance;
}

int main() {
	float* matrixHost = createMatrix(extN, extN);
	float* matrixDevice;
	
	int matrixSize = sizeof(float) * extN * extN;

	cudaMalloc((void**)&matrixDevice, matrixSize);
	cudaMemcpy(matrixDevice, matrixHost, matrixSize, cudaMemcpyHostToDevice);

	long startTime = clock();
	dim3 blocks(N / blockXSize, N / blockYSize);
	dim3 threads(blockXSize / tileXSize, blockYSize / tileYSize);

	for (size_t i = 0; i < ITERATIONS; i++) {
		calculateBlack <<< blocks, threads >>>(matrixDevice);
		cudaDeviceSynchronize();
		calculateRed <<< blocks, threads >>>(matrixDevice);
		cudaPeekAtLastError();
		cudaDeviceSynchronize();

    if(i%1000 == 0) {
        cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);
        float diff = calculateDifference(matrixHost);
        cout << " Difference: " << diff << "iter: " << i;
        if(diff < 0.01) break;
    }  
	}

	std::cout << "Calculation take time: " << clock() - startTime << std::endl;
  cout << " Difference: " << calculateDifference(matrixHost);
	cudaError_t cudaResult = cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

	if (cudaResult != cudaSuccess) {
		std::cout << "Cuda error: " << cudaResult << std::endl;
	}

	return 0;
}

KeyboardInterrupt: ignored

In [0]:
%%cu
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include "linux/kernel.h"
#include <iostream>
#include <ctime>
#include <iomanip>
#include <fstream>
#include <math.h>
#include <cmath>

using namespace std;

#define ITERATIONS 200000
#define N 256

#define e 2.7182818

#define blockZSize 3

#define blockXSize 32
#define blockYSize 32

#define tileXSize 4
#define tileYSize 4

#define extN (N + 2 * blockZSize)
#define Dx 1
#define h ((float) Dx / (N + 1))
#define h2 powf(h, 2.0)

__device__
float f(int i, int j) {
	return 2 * powf(e, i * h + j * h);
}

float u_t(float x) {
	return powf(e, x);
}

float u_l(float x) {
	return powf(e, x);
}

float u_r(float x) {
	return powf(e, x + 1);
}

float u_d(float x) {
	return powf(e, x + 1);
}


__device__
void tile(float* matrix, int iGl, int jGl, bool isBlack) {
	int iBegin = iGl * blockXSize + threadIdx.x * tileXSize + blockZSize;
	int jBegin = jGl * blockYSize + threadIdx.y * tileYSize + blockZSize;

  int N1 = tileXSize + blockZSize * 2;
  int N2 = tileYSize + blockZSize * 2;
  
	float helper[12 * 12];
    
  for(int i = 0; i < N1; i++) {
		for(int j = 0; j < N2; j++) {
			helper[i * N1 + j] = matrix[extN * (iBegin - blockZSize + i) + jBegin - blockZSize + j];
		}
	}
  
  int iSplit = iBegin - 2 * blockZSize + 1;
  int jSplit = jBegin - 2 * blockZSize + 1;
    
  for(int k = 1; k <= blockZSize; k++) {
		for (int i = k; i < N1 - k; i++) {          
   
		  for (int j = k; j < N2 - k; j += 1) {
        int b = N1 * i + j;
			  helper[b] = 0.25 * (
            helper[b + N1]
          + helper[b - N1]
          + helper[b + 1]
				  + helper[b - 1] 
          - h2 * f(iSplit + i, jSplit + j)
        );
		  }
	  }
	}

	for (int i = iBegin; i < iBegin + tileXSize; i++) {
    int offset = (i) % 2;
		if (isBlack) {
			  offset = 1 - offset;
		}
		for (int j = jBegin + offset; j < jBegin + tileYSize; j += 2) {
      int helperIndex = N1 * (i - iBegin + blockZSize) + j - jBegin + blockZSize;
			matrix[extN * i + j] = helper[helperIndex];
		}
	}
}

__global__
void calculateBlack(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, true);
}

__global__
void calculateRed(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, false);
}

float func(int i, int j) {
	return powf(e, i * h + j * h);
}

float* createMatrix(int row, int col) {
	float* matrix = new float[row * col];

	for(int i = 0; i < row; i++) {
		for(int j = 0; j < col; j++) {
			matrix[i * row + j] = 0;
		}
	}

  for(int k = 0; k < blockZSize; k++) {
    for(int i = k; i < row - k; i++) {
      matrix[k * row + i] = u_t((i - blockZSize + 1) * h);
      matrix[(row - 1 - k) * row + i] = u_d((i - blockZSize + 1) * h);
    }
	}

  for(int k = 0; k < blockZSize; k++) {
    for(int j = k; j < col - k; j++) {
      matrix[j * col + k] = u_l((j - blockZSize + 1) * h);
      matrix[j * col + row - 1 - k] = u_r((j - blockZSize + 1) * h);
	  }
  }
    
	return matrix;
}

float calculateDifference(float* matrix1) {
	float tolerance = 0.0f;
  int tolerancei = 0;
  int tolerancej = 0;
  for (int i = blockZSize; i < extN - blockZSize; i++) {
		for (int j = blockZSize; j < extN - blockZSize; j++) {
      float fCurrent = func(i - blockZSize + 1, j - blockZSize + 1);
      if(abs(matrix1[i * extN + j] - fCurrent) > tolerance) {
          tolerance = abs(matrix1[i * extN + j] - fCurrent);
          tolerancei = i;
          tolerancej = j;
      } 
		}
	}
  cout << " i: " << tolerancei << "j: " << tolerancej;
	return tolerance;
}

int main() {
	float* matrixHost = createMatrix(extN, extN);
	float* matrixDevice;
	
	int matrixSize = sizeof(float) * extN * extN;

	cudaMalloc((void**)&matrixDevice, matrixSize);
	cudaMemcpy(matrixDevice, matrixHost, matrixSize, cudaMemcpyHostToDevice);

	long startTime = clock();
	dim3 blocks(N / blockXSize, N / blockYSize);
	dim3 threads(blockXSize / tileXSize, blockYSize / tileYSize);

	for (size_t i = 0; i < ITERATIONS; i++) {
		calculateBlack <<< blocks, threads >>>(matrixDevice);
		cudaDeviceSynchronize();
		calculateRed <<< blocks, threads >>>(matrixDevice);
		cudaDeviceSynchronize();

      
    cudaPeekAtLastError();
      
    if(i%1000 == 0) {
        cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);
        float diff = calculateDifference(matrixHost);
        cout << " Difference: " << diff << "iter: " << i;
        if(diff < 0.01) break;
    }  
	}

	std::cout << "Calculation take time: " << clock() - startTime << std::endl;
  cout << " Difference: " << calculateDifference(matrixHost);
	cudaError_t cudaResult = cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

	if (cudaResult != cudaSuccess) {
		std::cout << "Cuda error: " << cudaResult << std::endl;
	}

	return 0;
}

' i: 255j: 255 Difference: 7.16282iter: 0 i: 155j: 155 Difference: 2.68542iter: 1000 i: 139j: 139 Difference: 1.62409iter: 2000 i: 135j: 135 Difference: 0.962912iter: 3000 i: 132j: 132 Difference: 0.570307iter: 4000 i: 131j: 131 Difference: 0.338048iter: 5000 i: 131j: 131 Difference: 0.200548iter: 6000 i: 132j: 132 Difference: 0.119136iter: 7000 i: 133j: 133 Difference: 0.0709519iter: 8000 i: 136j: 136 Difference: 0.0424712iter: 9000 i: 140j: 140 Difference: 0.025691iter: 10000 i: 258j: 258 Difference: 0.0161867iter: 11000 i: 258j: 258 Difference: 0.0161858iter: 12000 i: 258j: 258 Difference: 0.0161848iter: 13000 i: 258j: 258 Difference: 0.0161839iter: 14000 i: 258j: 258 Difference: 0.0161829iter: 15000 i: 258j: 258 Difference: 0.0161829iter: 16000 i: 258j: 258 Difference: 0.0161829iter: 17000 i: 258j: 258 Difference: 0.0161829iter: 18000 i: 258j: 258 Difference: 0.0161829iter: 19000 i: 258j: 258 Difference: 0.0161829iter: 20000 i: 258j: 258 Difference: 0.0161829iter: 21000 i: 258j: 25

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.9 GB  | Proc size: 143.6 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


CODE (AT LEAST NOW)

In [0]:
!sudo apt-get install lshw;
!sudo lshw -C display;

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libpci3 pciutils usbutils
The following NEW packages will be installed:
  libpci3 lshw pciutils usbutils
0 upgraded, 4 newly installed, 0 to remove and 127 not upgraded.
Need to get 721 kB of archives.
After this operation, 2,870 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpci3 amd64 1:3.5.2-1ubuntu1.1 [24.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 lshw amd64 02.18-0.1ubuntu6.18.04.1 [231 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 pciutils amd64 1:3.5.2-1ubuntu1.1 [257 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/main amd64 usbutils amd64 1:007-4build1 [208 kB]
Fetched 721 kB i

In [0]:
import os

f = open("example.txt", 'r')
i = 0

content = f.readlines()
for line in content:
  if i % 34 == 0:
    print('\n')
  line = line.split(' ')
  for el in line:
    print(el.rjust(14), end='')
  i += 1


FileNotFoundError: ignored